In [1]:
%%bash
cat placement.mzn

int: N_CLOUD_REGIONS;  % number of cloud regions
% set of 1..N_CLOUD_REGIONS: C_REGIONS;

int: N_ATTRIBUTES; %  number of capability related attributes
% set of 1..N_ATTRIBUTES: ATTRIBS;
array[1..N_ATTRIBUTES] of float: w_attributes; % weights of each attribute

int: N_UTILIZATION_METRICS; % number of dynamic capacity metrics of interest
% set of 1..N_UTILIZATION_METRICS: U_METRICS;
array[1..N_UTILIZATION_METRICS] of float: w_metrics; % weights of each capacity metric

int: cust_type;  % customer type, 0 = regular, 1 = silver, 2 = gold
int: N_VMS; % number of VMs in VNF
int: N_CAPACITY_METRICS; % number of metrics for cloud region capacity check

float: C_ALLOC_THRESHOLD; % allocation threshold for cloud
float: CUST_ALLOC_THRESHOLD; % allocation threshold for customer in cloud
float: AVG_CPU_UTILIZATION_THRESHOLD;
float: PEAK_CPU_UTILIZATION_THRESHOLD;

enum CUST_TYPES = { STANDARD, SILVER, GOLD };
enum ATTRIBUTES = { CORE_DC, DIRECT_CONN, MIN_GUARANTEE, SRIOV };
enum METRICS = { AVG_C

In [2]:
%%bash
cat gold.dzn

% enum CUST_TYPES = { STANDARD, SILVER, GOLD };
% enum ATTRIBUTES = { CORE_DC, DIRECT_CONN, MIN_GUARANTEE, SRIOV };
% enum METRICS = { AVG_CPU_UTILIZATION, PEAK_CPU_UTILIZATION };
% enum CLOUD_REGION_CAPACITY = {CPU_CLOUD, MEMORY_CLOUD};

N_CLOUD_REGIONS = 3;  % e-dc-1, core-dc-1, core-dc-2
N_ATTRIBUTES = 4;
N_UTILIZATION_METRICS = 2;
N_CAPACITY_METRICS = 2;
N_VMS = 3;
C_ALLOC_THRESHOLD = 0.65;
CUST_ALLOC_THRESHOLD = 0.95;
AVG_CPU_UTILIZATION_THRESHOLD = 0.7;
PEAK_CPU_UTILIZATION_THRESHOLD = 0.95;

cust_type = GOLD;
N_VNFS = 1;

w_metrics = [0.9, 0.1];
w_attributes = [0.1, 0.9, 0.05, 0.05]; 

capabilities = [| 0, 1, 1, 1
                | 1, 0, 1, 1
                | 1, 1, 0, 1 |];

cpu_utilization = [| 0.05, 0.9
                   | 0.1, 0.5
                   | 0.6, 0.8 |];

c_alloc_capacity = [| 600, 1200
                    | 1200, 2400
                    | 2400, 4800 |];

c_total_capacity = [| 1000, 2000
                    | 2000, 4000
                    | 4000, 8000 |];

c_all

In [3]:
%%bash
diff gold.dzn gold-no-min-guarantee.dzn

23c23
<                 | 1, 0, 1, 1
---
>                 | 1, 0, 0, 1


In [4]:
%%bash
diff gold.dzn gold-no-min-guarantee.dzn

23c23
<                 | 1, 0, 1, 1
---
>                 | 1, 0, 0, 1


In [5]:
%%bash
cat silver.dzn

% enum CUST_TYPES = { STANDARD, SILVER, GOLD };
% enum ATTRIBUTES = { CORE_DC, DIRECT_CONN, MIN_GUARANTEE, SRIOV };
% enum METRICS = { AVG_CPU_UTILIZATION, PEAK_CPU_UTILIZATION };
% enum CLOUD_REGION_CAPACITY = {CPU_CLOUD, MEMORY_CLOUD};

N_CLOUD_REGIONS = 3;  % e-dc-1, core-dc-1, core-dc-2
N_ATTRIBUTES = 4;
N_UTILIZATION_METRICS = 2;
N_CAPACITY_METRICS = 2;
N_VMS = 3;
C_ALLOC_THRESHOLD = 0.65;
CUST_ALLOC_THRESHOLD = 0.95;
AVG_CPU_UTILIZATION_THRESHOLD = 0.7;
PEAK_CPU_UTILIZATION_THRESHOLD = 0.95;

cust_type = SILVER;
N_VNFS = 2;

w_metrics = [0.9, 0.1];
w_attributes = [0.1, 0.9, 0.05, 0.05]; 

capabilities = [| 0, 1, 1, 1
                | 1, 0, 1, 1
                | 1, 1, 0, 1 |];

cpu_utilization = [| 0.05, 0.9
                   | 0.1, 0.5
                   | 0.6, 0.8 |];

c_alloc_capacity = [| 600, 1200
                    | 1200, 2400
                    | 2400, 4800 |];

c_total_capacity = [| 1000, 2000
                    | 2000, 4000
                    | 4000, 8000 |];

c_a

In [6]:
%%bash
diff silver.dzn silver-high-edge-load.dzn

26c26
< cpu_utilization = [| 0.05, 0.9
---
> cpu_utilization = [| 0.05, 0.96


In [7]:
!mzn-gecode -a placement.mzn gold.dzn

Solution: 
Cloud Region for vDNS = 2
Objective function value: 1002.639
Customer type: 3
----------


In [8]:
!mzn-gecode -a placement.mzn gold-no-min-guarantee.dzn

Solution: 
Cloud Region for vDNS = 2
Objective function value: 2.589
Customer type: 3
----------
Solution: 
Cloud Region for vDNS = 3
Objective function value: 3.0195
Customer type: 3
----------


In [9]:
!mzn-gecode -a placement.mzn silver.dzn

Solution: 
Cloud Region for vDNS = 2
Cloud Region for vFW = 1
Objective function value: 6.062
Customer type: 2
----------
Solution: 
Cloud Region for vDNS = 3
Cloud Region for vFW = 1
Objective function value: 6.4425
Customer type: 2
----------


In [10]:
!mzn-gecode -a placement.mzn silver-high-edge-load.dzn

Solution: 
Cloud Region for vDNS = 2
Cloud Region for vFW = 2
Objective function value: 5.278
Customer type: 2
----------
Solution: 
Cloud Region for vDNS = 3
Cloud Region for vFW = 2
Objective function value: 5.6585
Customer type: 2
----------
Solution: 
Cloud Region for vDNS = 3
Cloud Region for vFW = 3
Objective function value: 6.039
Customer type: 2
----------
